# Quickstart

In this guide, we will train a simple Convolutional Neural Network (CNN) on MNIST handwritten digits dataset. We will simulate a Federated Learning experiment between two collaborators, orchestrated by an aggregator, using the TaskRunner CLI interface.

OpenFL must be installed for this tutorial. Refer to the [installation guide](https://openfl.readthedocs.io/en/latest/installation.html).

In [ ]:
# Suppress TensorFlow warnings
%env TF_CPP_MIN_LOG_LEVEL=3

### Create a workspace

For brevity, we will reuse an existing workspace.

In [ ]:
!fx workspace create --prefix ./mnist_example --template keras_cnn_mnist
%cd ./mnist_example

### Initialize a Plan

This step builds an entire FL experiment plan, along with the initial set of parameters that will be used in the experiment.
We supply `localhost` as the aggregator address, for simulation purposes.

In [ ]:
!fx plan initialize -a localhost

### Create a certificate authority (CA)

OpenFL supports mTLS, which ensures secure communication between the collaborators and the aggregator. This step generates a certificate authority (CA) that will be used to sign the certificates of the collaborators. The CA is generated only once and can be reused for multiple experiments.

In [ ]:
!fx workspace certify

### Create an aggregator, and its key-pair

In [ ]:
!fx aggregator generate-cert-request --fqdn localhost
!fx aggregator certify --fqdn localhost --silent

### Create collaborators, and their key-pairs

We will name our first collaborator `bob`.

In [ ]:
!fx collaborator create -d 0 -n bob --silent
!fx collaborator generate-cert-request -n bob --silent

Certify `bob` with the CA.

> Note: This is a command that runs on the `aggregator` side. `bob`'s certificate signing request (CSR) is sent to the `aggregator`, which then signs the certificate with the CA. The signed certificate then has to be sent back to `bob`.

In [ ]:
!fx collaborator certify --request-pkg col_bob_to_agg_cert_request.zip --silent

And the second collaborator `charlie`.

In [ ]:
!fx collaborator create -d 1 -n charlie --silent
!fx collaborator generate-cert-request -n charlie --silent

Certify `charlie` with the CA.

> Note: This is a command that runs on the `aggregator` side. `charlie`'s certificate signing request (CSR) is sent to the `aggregator`, which then signs the certificate with the CA. The signed certificate then has to be sent back to `charlie`.

In [ ]:
!fx collaborator certify --request-pkg col_charlie_to_agg_cert_request.zip --silent

### Import CA signed certificates

This step imports the CA-signed certificates of the collaborator and the aggregator into the workspace.

> Note: This command runs on respective collaborator sites.

In [ ]:
!fx collaborator certify --import agg_to_col_bob_signed_cert.zip
!fx collaborator certify --import agg_to_col_charlie_signed_cert.zip

### Start the simulation

This step starts the simulation of the FL experiment. The aggregator will orchestrate the training process between the collaborators.

In [ ]:
!fx aggregator start & fx collaborator start -n bob & fx collaborator start -n charlie